In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
files={}
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        files[filename] = pd.read_csv(os.path.join(dirname, filename))
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=files['train.csv']

In [ ]:
train_len=df.shape[0]

In [ ]:
df=pd.concat([df,files['test.csv']],axis=0).reset_index()

In [ ]:
columns=df.columns
print(columns)

In [ ]:
df.isnull().sum()

In [ ]:
#First Lets See embarked
df[df['Embarked'].isnull()]

In [ ]:
df['Embarked'].unique()

In [ ]:
df['Embarked'].fillna('Not',inplace=True)

In [ ]:
df['Embarked']=df['Embarked'].astype('category').cat.as_ordered()

In [ ]:
df['Embarked'].dtype

In [ ]:
df['Embarked'].isnull().sum()
df['Cabin'].unique()

In [ ]:
df['Null Cabin'] = False
df['Null Cabin'].loc[df['Cabin'].isnull()]=True
df['Cabin'].fillna('No Cabin',inplace=True)
df['Block']=df['Cabin'].str[0]
df['floor']=df['Cabin'].str[1]
df['floor'].fillna(0,inplace=True)


In [ ]:
df

In [ ]:
df.isnull().sum()

In [ ]:
bins=[0,15,25,35,45,55,65,100]

In [ ]:
pd.cut(df['Age'],bins,labels=['Child',"Teen","Young","early thirties","old experienced","old","really old"]).isnull().sum()

In [ ]:
df['Age_Bin'] = pd.cut(df['Age'],bins,labels=['Child',"Teen","Young","early thirties","old experienced","old","really old"]).cat.codes

In [ ]:
df['Age_Bin']

In [ ]:
fare_bin=[0,30,60,100,200,300,600]
df['Fare'] = pd.cut(df['Fare'],bins)

In [ ]:
df.info()

In [ ]:
cat_cols=list(df.columns[6:-1])
cat_cols.append('Sex')
cat_cols.append('Pclass')
cat_cols.append('Age_Bin')

In [ ]:
cat_cols

In [ ]:
X=df[cat_cols]

In [ ]:
X.info()

In [ ]:
X=X.drop(['Ticket','Age'],axis=1)


In [ ]:
cat_cols=X.columns

In [ ]:
cat_cols

In [ ]:
for c in cat_cols:
    X[c]=X[c].astype('category').cat.as_ordered().cat.codes

In [ ]:
X.info()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X_train=X.head(891)
Y=df['Survived']

In [ ]:
Y_train=Y.head(files['train.csv'].shape[0])

In [ ]:
X_train.corrwith(Y_train)

In [ ]:
X_pred=X.tail(files['test.csv'].shape[0])


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X_train,Y_train,test_size=0.33
                                              )

In [ ]:
model=RandomForestClassifier(n_jobs=-1)

In [ ]:
model.fit(X_train,Y_train)

In [ ]:
model.score(X_train,Y_train)

In [ ]:
model.score(X_test,Y_test)

In [ ]:
preds=model.predict(X_pred)

In [ ]:
preds

In [ ]:
d={
    'PassengerId':df.tail(files['test.csv'].shape[0])['PassengerId'],
    'Survived': preds
}
pred_df=pd.DataFrame(d,columns=['PassengerId','Survived'])

In [ ]:
pred_df['Survived']=pred_df['Survived'].astype(int)

In [ ]:
pred_df

In [ ]:
pred_df.to_csv('output.csv',index=False)